<a href="https://colab.research.google.com/github/Phubordin/portfolio-project/blob/main/portfolio-project/P09-Chatbot-Development-for-a-Pizza-Business-Using-R-Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# คำแนะนำ

ฝากเว็บไซต์ [My Portfolio Website (⭐️ Project 9)](https://phubordin.github.io/My-Portfolio-Website/project_rfm_py_dsb10.html)

---

ถึงทุกคนที่กำลังดูหน้าเว็บนี้อยู่บน [Google Colab](https://colab.research.google.com/drive/1r5JBCZ9OCMExucj47YEKTu8Utino1N5E?usp=sharing) สามารถลองสั่งอาหารได้ โดยกด

`File → Save a copy in Drive` เมื่อ Copy สำเร็จสามารถ

กด `Run All ใช้เวลารัน <30 วิ` ได้ที่มุมบนซ้ายได้เลย (`ต้องรันใน notebook ที่ copy ไปนะครับไม่งั้นจะรันไมไ่ด้`)

⭐️ ระบบจะเลื่อนลงไปข้างล่าง ให้เราพิมพ์ตามคำแนะนำได้เลย

แนะนำครั้งแรกให้พิมว่า `"menu"` เพื่อดูว่าเมนูราคาร้าน Pizza มีเมนูอะไรบ้าง

หวังว่าจะลองเล่นกันดูนะ ขอบคุณครับ. 🎉

# ติดตั้ง R Magic
สามารถรันโค้ด R ได้โดยไม่ต้องเปลี่ยนประเภทรันไทม์ แค่พิมพ์ %%R ไว้กับ cell ที่ต้องการจะรันโค้ด

(ที่ผมไม่เปลี่ยนประเภทรันไทม์เป็น R เพราะการแสดงผลข้อความแนะนำไม่เหมือนบน RStudio หรือ Posit Clod)

ขออนุญาตละไว้ตรงนี้นะครับ

In [ ]:
# ติดตั้ง rpy2 (ต้องรันเสมอเมื่อเปิด notebook ใหม่)
!pip install -q rpy2

# โหลด R magic โหลดเพื่อไว้รัน R ได้โดยไม่ต้องเปลี่ยนประเภท Runtime
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


# โครงสร้างกระบวนการสั่ง Pizza Chatbot ด้วยภาษา R

สร้าง Workflow ต่างๆ while, if รวมถึงกำหนดค่าแสดงข้อความต่างๆ

In [ ]:
# Syntax เปลี่ยนรันไทม์เป็น R Runtime เฉพาะ Cell นี้ทั้งหมด
%%R

# กำหนดรายชื่อเมนูอาหารและเครื่องดื่มต่างๆ ไว้ใน Vector ชื่อ Menu (เอาไว้แสดงรายชื่อ menu อาหารที่ลูกค้าพิมพ์ check เข้ามา)
menu <- c(
  "Margherita Pizza", "Pepperoni Pizza",
  "BBQ Chicken Pizza", "Vegetarian Pizza",
  "Seafood Deluxe Pizza",
  "Coca-Cola", "Iced Lemon Tea",
  "Orange Juice", "Sparkling Water",
  "Mango Smoothie",
  "Chocolate Lava Cake", "Tiramisu",
  "Cheesecake", "Fruit Salad",
  "Ice Cream Sunday"
)

# ปรับรายชื่อเมนูอาหารเครื่องดื่มเป็นตัวพิมพ์เล็กให้หมด และเก็บไว้ในตัวแปร Vector ใหม่ชื่อ menu_lower (เอาไว้ verify ตอนที่ users สั่งอาหารเข้ามา)
menu_lower <- tolower(menu)

# กำหนดราคาของแต่ละเมนูตามลำดับเดียวกันกับ vector เมนู ด้วยการสุ่มตัวเลข ผลลัพธ์จะเป็น vector ตัวเลข
price <- c(
  sample(15:20, 5),  # สุ่มราคาพิซซ่า แต่ละถาดให้มีช่วงราคาอยู่ที่ $15-$20 มาทั้งหมด 5 ถาด
  sample(2:6, 5),    # สุ่มราคาเครื่องดื่ม แต่ละแก้วให้มีช่วงราคาอยู่ที่ $2-$6 มาทั้งหมด 5 แก้ว
  sample(7:12, 5)    # สุ่มราคาของหวาน แต่ละถ้วยให้มีช่วงราคาอยู่ที่ $7-$12 มาทั้งหมด 5 ถ้วย
)

# สร้างตารางข้อมูล โดยเอา vector ก่อนหน้ามาใส่ นั่นคือ menu, price มารวมกันเป็นตารางข้อมูล พร้อมตั้งชื่อคอลัมน์ใหม่
df_menu <- data.frame(MENU = menu, PRICE = price)


# เริ่มสร้างโครงสร้างการทำงานของ Chatbot สั่ง pizza อาหาร และเครื่องดื่มต่างๆกันเลย
# สร้างฟังก์ชัน pizza() โดยที่เมื่อเรียกใช้ pizza() ขึ้นมาให้..
pizza <- function() {
  cat("\nWelcome to the Wonderful Pizza Shop!\n") # แสดงผล ยินดีต้อนรับสู่ร้าน Wonderful Pizza

  collect_order <- list()  # สร้าง empty list มาเพื่อรอรับค่าจากที่ users พิมพ์มา (ในที่นี้คือ เมนูเครื่องดื่มต่างๆที่ลูกค้าจะสั่งเข้ามา)

  # สร้าง while ให้รันอย่างต่อเนื่อง
  while (TRUE) {
    # แสดง คำแนะนำต่างๆ ที่ users จำเป็นต้องรู้ !
    cat("Order now! :\n") # แสดงผล เริ่มสั่งอาหาร !
    cat("Type 'menu' to see the menu,\n") # ลูกค้าพิมพ์ menu เพื่อที่จะดูว่าร้านเรามีเมนูอะไรบ้าง ราคาแต่ละเมนูเป็นยังไง
    cat("Type 'check' to view your basket and check the bill,\n") # ลูกค้าพิมพ์ check เพื่อตรวจสอบว่าในตระกร้ามีเมนูอะไรแล้วบ้าง
    cat("Type 'reorder' to clear your basket and start over,\n") # ลูกค้า reorder เพื่อที่จะล้างตระกร้าให้หมด เพื่อที่จะเพิ่มของลงตระกล้าใหม่ (ไม่มีให้ ลด/เพิ่ม จำนวนเมนูในขณะสั่ง ต้อง reorder ใหม่อีกรอบ)
    cat("Type 'stop' to leave the restaurant.\n") # ลูกค้าพิมพ์ stop เพื่อ (checkout, หากไม่สนใจเมนูใดๆและยังไม่มีเมนูในตระกร้า ก็สามารถยกเลิกการสั่งได้โดยการพิมพ์ stop เพื่อหยุดการทำงานของ chatbot)

    # สร้างกล่องข้อความสำหรับให้ users พิมพ์โต้ตอบกับตัว Chatbot พร้อมแสดงข้อความถาม users ว่า "คุณชอบเมนูไหนเป็นพิเศษไหม?"
    order_item <- tolower(readline("What would you like to do? "))

    # เริ่มกระบวนการตรวจสอบว่า สิ่งที่ users พิมพ์เข้ามา เราจะให้แสดงผลอะไรออกไปบ้าง ซึ่งสิ่งที่ users อาจพิมพ์คือ ข้อความคำแนะนำก่อนหน้าคือ (menu, check, reorder, stop, others..)
    if (order_item %in% "menu") { # ถ้า users พิมพ์ menu
      cat("\n----------- MENU -----------\n") # ให้แสดงผล หัวข้อใหญ่ชื่อว่า MENU
      print(df_menu) # แสดงตารางรายการชื่อเมนูพิซซ่า อาหาร และเครื่องดื่ม รวมถึงราคา
      next # เมื่อแสดงผลเสร็จแล้วให้ ดำเนินการต่อไป โดยจะกลับไปรันโค้ดก่อนหน้า กล่าวคือ (ให้แสดงข้อความว่า "What would you like to do?" และให้ users พิมพ์เข้ามาหลังจากดูเมนูเสร็จแล้ว)
    }

    if (order_item %in% "stop") { # แต่ถ้า users พิมพ์ stop กล่าวคือ users อาจไม่สนใจรายการอาหารของเรา ก็จะ..
      break # หยุดกระบวนการโต้ตอบ Chatbot ทันที (คำสั่งสือไม่สำเร็จ ยกเลิก order ในตระกร้าทั้งหมด)
    }

    if (order_item %in% "check") { # แต่ถ้าลูกค้าสั่งเข้ามาเรื่อยๆ แล้วอยู่ดีๆอยาก check ว่าสั่งอะไรไปแล้วบ้างโดยการพิมพ์คำว่า check เข้ามาระบบก็จะทำการ..
      if (length(collect_order) == 0) { # ตรวจสอบว่ามีเมนูถูกสั่งเข้ามาหรือยังไง ถ้ายังไม่มีให้แสดงว่า..
        cat("\nYour basket is empty.\n") # "ตระกร้าคุณยังว่างอยู่นะครับ" (Your basket is empty.)
      } else { # แต่ถ้าในตระกร้ามีเมนูอาหารอยู่
        # ให้เอาชื่อเมนูแต่ละเมนูในตระกร้า (names(collect_order))
        # ไปจับคู่กับชื่อเมนู(lowercase) เราจะได้ตำแหน่งออกมาพร้อมนำไป subset by position
        original_menu_names <- sapply(names(collect_order), function(name) {
          # เช่น menu[1] = Margherita Pizza เลข 1 คือตำแหน่ง (เพราะเราอยากให้ users เห็นชื่อเมนูแบบ Capitalized ถ้าเราเอา names(collect_order) มามันจะเป็นตัว lowercase ทั้งหมด)
          # ดังนั้นเวกเตอร์ original_menu_names จะเก็บคือเมนูอาหารแบบ Capitalized ไว้ในค่า และจะมีชื่อเมนูแบบ lowercase ไว้กำกับแต่ละค่าในเวกเตอร์นี้อีกที เราค่อยนำออกตอนนำไปสร้างตารางสรุปรายการคำสั่งซื้อ
          menu[match(name, menu_lower)]
        })

        # สร้างตารางสรุปคำสั่งซื้อ ให้ลูกค้าเห็น
        order_summary <- data.frame(
          # คอลัมน์ "Menu" คือชื่อเมนูจริง (Original names แบบ Capitalized)
          Menu = unname(original_menu_names), # sapply() เมื่อประมวลผลเสร็จ มันจะ “คืนค่าผลลัพธ์เป็นเวกเตอร์” และ “คงชื่อเดิมของ input” เอาไว้เป็น names ของเวกเตอร์ผลลัพธ์ ดังนั้นเราจึงใช้ unname เพื่อนำชื่อออก

          # คอลัมน์ "Quantity" คือจำนวนที่สั่ง โดยแปลง collect_order เป็นเวกเตอร์ (unlist) จากนั้นนำชื่อแต่ละ argument ออกเพราะมันคือชื่อเมนูเราไม่ต้องการ เราต้องการแค่จำนวนแต่ละเมนูที่ลูกค้าใส่เข้ามาในตระกร้าเท่านั้น
          Quantity = unname(unlist(collect_order)),

          # คอลัมน์ "Unit_Price_Dollar" คือราคาต่อหน่วยของเมนู, ใช้ match() เพื่อหา index ของชื่อใน menu_lower แล้วดึงราคาจาก price
          Unit_Price_Dollar = price[match(names(collect_order), menu_lower)],

          # คอลัมน์ "Total_Price" คือราคารวม (Quantity * Unit Price) ของแต่ละเมนูที่อยู่ในตระกร้าที่ลูกค้าสั่ง โดยนำ..
          # list แปลงเป็นเวกเตอร์เพื่อจะนำ Quantity ออกมา และนำไปคูณกับ price ที่เกิดจากการ sutset position โดยนำชื่อของ list (ชื่อเมนูที่อยู่ในตระกร้า) ไปจับคู่กับชื่อเมนูตัวพิมพ์เล็กทั้งหมดเพื่อจะได้ระบุตำแหน่งของราคาต่อหน่วยที่ถูกต้อง และดึง price[] มาได้ถูกต้อง
          # เมื่อคูณกันจะมีชื่อ กำกับค่าต่างๆในเวกเตอร์ให้เรานำออก ก่อนนำไปสร้างตารางสรุปผลลัพธ์โดยใช้ unname ทีหลัง
          Total_Price_Dollar = unname(unlist(collect_order) * price[match(names(collect_order), menu_lower)]),

          # ไม่ให้ R แปลงคอลัมน์ประเภทข้อความ กลายเป็นประเภท factor
          stringsAsFactors = FALSE
        )

        # แสดงตารางผลลัพธ์ พร้อมข้อความระบุ
        cat("\nYour current order summary:\n") # แสดงข้อความระบุ
        print(order_summary) # แสดงตารางสรุปคำสั่งซื้อ

        # สรุปจำนวนเงินที่ลูกค้าต้องจ่ายทั้งหมด นำค่าจากคอลัมน์ Total_Price_Dollar มารวมกันทั้งหมด
        total_order <- sum(order_summary$Total_Price_Dollar)
        cat("\nOrder Total: $", total_order, "\n", sep = "") # แสดงข้อความ (sep = "" หมายถึง แบ่งโดยค่าว่าง) เช่น "Order Total: $18"

        # จากนั้นถามลูกค้าว่า อยากยืนยันคำสั่งซื้อเลยไหม โดยให้ตอบ Y หรือ N? ต่อให้ลูกค้าจะพิมพ์แบบ lowercase หรือ uppercase เราจะตั้งค่าให้เป็นตัว lowercase ทั้งหมด
        confirm_order <- tolower(readline("\nDo you confirm your order? (Y/N): "))

        # ถ้าลูกค้าตอบ Y เราจะแสดงข้อความ "ยืนยันคำสั่งซื้อ! ขอบคุณที่ใช้บริการ" พร้อมกับแสดงตาราง(คล้ายใบเสร็จบอกว่าลูกค้าได้สั่งซื้ออะไรไป)
        if (confirm_order %in% "y") {
          cat("\nOrder confirmed! Thank you for your purchase.\n")
          return(order_summary)
        } else if (confirm_order %in% "n") {  # แต่ถ้าลูกค้าตอบ N เราจะแสดงข้อความบอกลูกค้าว่า "คุณลูกค้าสามารถเริ่มสั่งซื้อต่อได้เลยครับ"
          cat("\nYou can continue ordering.\n")
          next #  พร้อมกระโดดไปรันโค้ดบรรทัดที่เราถามลูกค้าว่า "คุณชอบเมนูไหนเป็นพิเศษไหม?" (ตรง while(TRUE) ที่แสดงคำแนะนำให้ลูกค้า)
        } else { # แต่ถ้าลูกค้าไม่ได้พิมพ์ Y หรือ N เราจะแสดงข้อความ "ใส่ข้อมูลไม่ถูกต้อง ให้กลับไปหน้าเมนูใหม่อีกครั้ง"..
          cat("Invalid input. Returning to menu.\n")
          next # (ก็คือหน้าที่แสดงคำแนะนำให้ลูกค้า พร้อมถามลูกค้าใหม่ว่า "คุณชอบเมนูไหนเป็นพิเศษไหม?" นั่นเอง)
        }
      }
      next # แสดงคำแนะนำใหม่ให้ลูกค้าทราบอีกครั้ง พร้อมถามลูกค้าใหม่ว่า "คุณชอบเมนูไหนเป็นพิเศษไหม?" (โค้ดบรรทัดนี้จะทำงานก็ต่อเมื่อในตระกร้า นั้นว่างเปล่า
    }

    # ถ้าลูกค้าพิมพ์ว่า "reorder" ระบบจะทำการ..
    if (order_item %in% "reorder") {
      cat("\nYou chose to clear your basket and start over.\n") # แสดงข้อความ "คุณได้เลือกล้างตระกร้าของคุณให้ว่างเปล่าเรียบร้อยแล้ว"
      collect_order <- list() # ตระกร้าหรือ list ที่เก็บข้อความจะว่างเปล่า
      next # พร้อมแสดงคำแนะนำใหม่ให้ลูกค้าทราบอีกครั้ง พร้อมถามลูกค้าใหม่ว่า "คุณชอบเมนูไหนเป็นพิเศษไหม?"
    }

    # ถ้าลูกค้าพิมพ์ ชื่อเมนูอาหารต่างๆที่มีอยู่ในเวกเตอร์ menu_lower ระบบจะทำการ..
    if (order_item %in% menu_lower) { # ถามลูกค้าต่อเนื่องว่า.. "ต้องการเมนูอาหารที่สั่งเมื่อสักครู่นี้กี่ชุดดีครับ?"
      order_quantity <- as.integer(readline(paste("How many", order_item, "would you like to order? "))) # โดยใช้ paste() เชื่อมคำ, readline() รับ input จาก users, as.integer() แปลงเป็นตัวเลขจำนวนเต็ม

      # เมื่อรับค่าจากที่ users พิมพ์เข้ามาแล้ว (จำนวนของชุดเมนูที่สั่ง) เราจะมา verify ว่า.. สิ่งที่ users พิมพ์มานั้นเป็นตัวเลขที่สอดคล้องไหม?
      if (is.na(order_quantity) || order_quantity <= 0) { # ถ้า users พิมพ์มาเป็นค่า NA หรือจำนวนเต็มลบ หรือจำนวนเต็มศูนย์
        cat("Invalid quantity. Please enter only a positive integer!\n") # ให้แสดงข้อความตอบกลับว่า "จำนวนชุดเมนูไม่ถูกต้อง กรุณาระบุตัวเลขจำนวนเต็มบวกเท่านั้น"
        next # พร้อมแสดงคำแนะนำใหม่ให้ลูกค้าทราบอีกครั้ง พร้อมถามลูกค้าใหม่ว่า "คุณชอบเมนูไหนเป็นพิเศษไหม?"
      }

      if (order_item %in% names(collect_order)) { # แต่ถ้าเกิดเหตุการณ์ที่ในตระกร้านี้มีเมนูนี้อยู่แล้ว (ลูกค้าใส่ชื่อเมนู แล้วจำนวนชุดของเมนูนี้มาใหม่) ระบบจะทำการ..
        confirm_add <- tolower(readline("You have already ordered this item. Do you want to add more? (Y/N): ")) # ตอบกลับว่า "ลูกค้ามีคำสั่งซื้อเมนูนี้อยู่แล้ว คุณต้องการจะเพิ่มจำนวนชุดของเมนูนี้อีกรอบไหม? ให้ตอบ Y หรือ N"

        if (confirm_add %in% "y") { # ถ้า users ตอบ Y (ยอมรับว่าจะเพิ่มจำนวนชุดของเมนูนี้อีก..)
          collect_order[[order_item]] <- collect_order[[order_item]] + order_quantity # เราจะอัพเดทจำนวนชุดของเมนูนี้ในตระกร้า ถ้าหากเดิมมีอยู่ 2 แล้วลูกค้าพิมพ์ชื่อเมนูนี้มาพร้อมกับระบุจำนวนมาอีกครั้งคือ 3 เราจะนำชุดเมนูนี้เป็น 2+3 นั่นคือ 5
          cat("Updated:", order_item, "-", collect_order[[order_item]], "in total.\n") # พร้อมเราจะแสดงข้อความบอกลูกค้าว่า "อัพเดทชุดเมนูของคุณแล้ว เช่น "Updated: cheesecake - 11 in total." (กรณีลูกค้าสั่ง cheesecake ซ้ำ)
        } else if (confirm_add %in% "n") { # ถ้า users ตอบ N (ปฎิเสธว่าจะไม่เพิ่มเมนูชุดนี้อีก) ระบบจะทำการ..
          next # แสดงคำแนะนำใหม่ให้ลูกค้าทราบอีกครั้ง พร้อมถามลูกค้าใหม่ว่า "คุณชอบเมนูไหนเป็นพิเศษไหม?"
        } else { # หรือถ้า users ไม่ได้พิมพ์มาว่า Y หรือ N เป็นอื่นๆนอกจาก 2 ตัวนี้ ระบบจะทำการ..
          cat("Invalid input. Skipping this item.\n") # แสดงข้อความว่า "ค่าที่รับเข้ามาไม่ถูกต้อง. ข้ามการเพิ่มเมนูนี้ไปพร้อมรับออเดอร์ใหม่อีกครั้ง"
          next # พร้อมแสดงคำแนะนำใหม่ให้ลูกค้าทราบอีกครั้ง พร้อมถามลูกค้าใหม่ว่า "คุณชอบเมนูไหนเป็นพิเศษไหม?"
        }
      } else { # ต่อเนื่องจากค่าที่รับจาก users (ถ้าหากเป็นจำนวนเต็มบวกแล้ว) ระบบจะทำการ..
        collect_order[[order_item]] <- order_quantity # บันทึกจำนวนชุดเมนูอาหารเครื่องดื่ม ที่ users สั่งเข้ามาพรอ้มกับ ระบุว่า..
        cat("Added:", order_item, "->", order_quantity, "\n") # "เมนูนี้ ถูกเพิ่มเข้าแล้ว จำนวนหนึ่ง" เช่น "Added: cheesecake -> 1" (กรณีลูกค้าสั่งเมนู cheesecake 1 EA)
      }
    } else { # 📌🔥🔥 สุดท้ายก่อนออกจาก while(TRUE) ถ้าหากไม่เข้าเงื่อนไขใดๆเลย เช่น ลูกค้าไม่ได้พิมพ์ว่า "menu", "check", "reorder" หรือ "stop" ระบบจะทำการ..
      cat("Invalid input. Please try again.\n") # แสดงข้อความตอบกลับว่า "ค่าที่รับไม่ถูกต้อง. โปรดลองใหม่อีกครั้ง" จะขึ้นคำถามใหม่ว่า "คุณชอบเมนูไหนเป็นพิเศษไหม?" (เพราะเรายังอยู่ใน while(TRUE) อยู่จะออกจากการใส่ค่าได้ต้องพิมพ์ "stop" หรือยืนยันคำสั่งซื้อแล้ว เท่านั้น)
    }
  }

  if (length(collect_order) == 0) { # สมมติว่า ถ้า users ยืนยันคำสั่งซื้อไปแล้วก็จะจบไป 1 กรณี, แต่พิมพ์ว่า users เพิ่มเมนูเข้าตระกร้าหรือไม่มีเมนูใดๆอยู่ในตระกร้าเลย และ users พิมพ์ว่า "stop" ระบบจะทำการ..
    print("Your basket is currently empty. Thank you for visiting and using our service!") # แสดงข้อความว่า "ตระกร้าของคุณยังไม่มีเมนูใดๆ ขอบคุณที่ใช้บริการนะครับ" กรณีที่ตระกร้าของคุณลูกค้าว่างเปล่า แล้วถ้า..
  } else { # ถ้าหากมีเมนูอยู่ในตระกร้า หลังจากที่ users พิมพ์ว่า "stop" ระบบจะตอบกลับก่อนจะจบการสั่งอาหารร้าน pizza ว่า..
    # แสดงข้อความ "หากคุณลูกค้ากลับมาใหม่ เมนูคุณในตระกร้าจะหายไปนะครับ โอกาสหน้าเราเรียนเชิญและยินดีต้อนรับคุณลูกค้านะครับ"
    print("If you return for a new visit, the items in your basket will no longer be there. We warmly invite you and look forward to welcoming you again!")
  }
}


# โค้ดเริ่มสั่ง Pizza พร้อมบันทึกเมนู

In [ ]:
# Syntax เปลี่ยนรันไทม์เป็น R Runtime เฉพาะ Cell นี้ทั้งหมด
%%R

# รันโค้ด Cell นี้ เพื่อเริ่มสั่งพิซซ่า หรือเมนูเครื่องดื่มต่างๆ ได้เลย! พร้อมบันทึกข้อมูลไว้ใน Order
order_summary <- pizza()



Welcome to the Wonderful Pizza Shop!
Order now! :
Type 'menu' to see the menu,
Type 'check' to view your basket and check the bill,
Type 'reorder' to clear your basket and start over,
Type 'stop' to leave the restaurant.
What would you like to do? stop
[1] "Your basket is currently empty. Thank you for visiting and using our service!"


# แสดง/บันทึก เมนู Pizza ที่ลูกค้าสั่งล่าสุด

เก็บข้อมูลตารางไว้ในตัวแปรชื่อ `order_summary` เผื่อเราจะนำตารางนี้ไปวิเคราะห์ต่อ


In [ ]:
# Syntax เปลี่ยนรันไทม์เป็น R Runtime เฉพาะ Cell นี้ทั้งหมด
%%R

# แสดงผลการกระทำที่ลูกค้าสั่งอาหารเมื่อสักครู่ ไม่ว่าจะเป็น..
# - สั่งอาหารสำเร็จ (แสดงเมนูรายการอาหารที่สั่ง)
# - ออกจากร้านโดยพิมพ์ "stop" ตั้งแต่ยังไม่มีหรือมีเมนูในตระกร้า (แสดงข้อความขอบคุณคุณลูกค้า)
order_summary


[1] "Your basket is currently empty. Thank you for visiting and using our service!"
